# (modified version: ignores lumen) Filter and Analyze CellProfiler Database Files for SOSR-COs Data

Andrew Tidball Lab, University of Michigan School of Medicine Department of Neurology

**Notebook Summary**

* reads in CellProfiler database file
* Remove all columns that start with "GFP_Core"
* reads in metadata and welldata
* Remove organoids (each row is an organoid) that have NaN values for (organoid area)
* merge in the metadata and welldata
* one-way ANOVA and Kruskal test on the organoid area for each concentration, (did NOT filter for lumen number, and did NOT filter for area yet)
* lists of (organoid area) for each concentration+condition (did NOT filter for lumen number, and did NOT filter for area yet) to excel file called (databaseFile)_Area_SOSRSonly.xlsx
* Filter out the organoids based on organoid area
* write the filtered file to an Excel file called (databaseFile)_filtered_SOSRSonly.xlsx. It is ready to be sent to the heatmap_final.ipynb Jupyter Notebook file to graph heatmaps and concentric maps
* Center and scale the desired columns for all of the conditions. You can customize which columns are getting centered+scaled. Write the filtered+centeredScaled combined-conditions file to an Excel file called (databaseFile)_centeredScaled_combined_SOSRSonly.xlsx
* Center and scale the desired columns for each unique condition. You can customize which columns are getting centered+scaled. Write the filtered+centeredScaled file for each unique condition to separate Excel files called (databaseFile)\_centeredScaled\_(conditionName)_SOSRSonly.xlsx

**How to Use**
* edit the cell named "User Input Edit Required"
* Kernel -> Restart and Run All

**Result**
* 4 types of excel files to a specified folder
* (databaseFile)_Area_SOSRSonly.xlsx
* (databaseFile)_filtered_SOSRSonly.xlsx
* (databaseFile)_centeredScaled_combined_SOSRSonly.xlsx
* (databaseFile)\_centeredScaled\_(conditionName)_SOSRSonly.xlsx (we will have N of these files, where N is the number of unique conditions we have)

**Note**
* this file is **a modified copy of the "Filter and Analyze - original" Jupyter Notebook file**. In this file, we remove all the GFP_Core columns, don't do any filtering and analysis based on (number of lumens), and do not calculate for the area ratio
* there are some old database files that did not have MyExpt_Per_object table in the SQL database files. this Jupyter notebook file does not support those database files
* this Jupyter notebook file does not support database files that have multiple plates all combined in one file

**Import libraries**: here, we import Python libraries that allow us to efficiently run statistical analysis, plot graphs, etc

In [ ]:
#Imports
import pandas as pd
import seaborn as sb
import numpy as np
import os
import sqlite3
import scipy
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler

**User Input Edit Required:**
* set the **databaseFile** variable to the name of the database file you want to read from
* set the **folder** variable to the name of the folder that you want to save the excel files to
* set the **metadataFile** variable to the name of the metadata file you want to read from
* set the **metadataSheet** variable to the name of the specific sheet of the metadataFile

In [ ]:
##USER INPUT##
databaseFile = "SLFA-093"
folder = "Results_SOSRSonly"
metadataFile = "Well_vs_Cond - 96 well plate SLFA93.xlsx"
metadataSheet = "Well_vs_Cond - 96 well plate sc"

**Retrieve the database SQL file**:
1. Next, we connect to the databaseFile's SQL database using the SQLAlchemy library by creating an engine called cnx
2. We use panda's read_sql_table function to read from the cnx engine. We are reading in the "MyExpt_Per_Object" table, and storing it in a pandas dataframe called **df**
* uncomment display(df) to get an overview of what df currently looks like
* uncomment df.to_excel("Raw.xlsx") to get an excel sheet of the original database file
* uncomment the last two lines to get a list of available table names in the SQL engine


In [ ]:
# SQLAlchemy connectable
cnx = create_engine('sqlite:///' + databaseFile + ".db").connect()

# table will be returned as a dataframe.
df = pd.read_sql_table('MyExpt_Per_Object', cnx)

#display(df)
#df.to_excel("Raw.xlsx")

##uncomment below if you want a list of table names
#table_list = [a for a in cnx.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
#print(table_list)




**Remove all columns that start with "GFP_Core"**
1. store all columns with names that include 'GFP_Core' in **gfp_cols** list
2. In **df**, drop the columns mentioned in **gfp_cols**

In [ ]:
gfp_cols = df.columns[df.columns.str.contains(pat='.*GFP_Core.*')].tolist()
display(gfp_cols)
df = df.drop(columns=gfp_cols)
display(df)

for c in df.columns:
    print(c, df[c].dtype)

**Read in metadata and well data**:
1. Use panda's read_excel function to read in the metadata file. If there are multiple sheet tabs, specify which sheet tab you want to read in "sheet_name = ". It reads in the file and stores it in a panda's dataframe called **metadata**
2. Next, we use panda's read_sql_table function to read from the cnx engine. We are reading in the "MyExpt_Per_Image" table, and storing it in a pandas dataframe called **well**
3. well is a big file, but we are only interested in two of its columns. Here, we are extracting out the "ImageNumber" and "Image_Metadata_WellID" columns. Now, the **well** dataframe just has these two columns.
* note: can use display(well) or display(metadata) to get an overview of those dataframes
* uncomment well.to_excel("Well.xlsx") to get an excel sheet of the well data

In [ ]:
metadata = pd.read_excel(metadataFile, sheet_name = metadataSheet)

well = pd.read_sql_table('MyExpt_Per_Image', cnx)

well = well[["ImageNumber", "Image_Metadata_WellID"]]

display(metadata)
display(well)
#well.to_excel("Well.xlsx")


**Remove organoids that have NaN values for (organoid area)**
1. We don't want the NaN values for (organoid area). In **df**, we drop a row if its  'spheroids_AreaShape_Area' is NaN.

**df** now has:
* filtered out NaN values for (organoid area)

In [ ]:
## prints out all column names and their value type 
#for c in df.columns:
#    print(c, df[c].dtype)
    
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

df = df.dropna(subset=['spheroids_AreaShape_Area'])


**Adding the metadata and well information**
1. Use panda's merge function to merge two dataframes: **df** and **well**. Their shared column is "ImageNumber" 
2. **df** now has a new column called 'Image_Metadata_WellID' at the end. We are renaming that column to 'WellID'
3. Use panda's merge function to merge two dataframes: **df** and **metadata**. Their shared column is "WellID" 
4. note: if you want to export the current **df** dataframe to excel, use df.to_excel("after_merge.xlsx"). you can change the file name
5. note: panda's merge can merge two dataframes in different ways. It's important to understand how it's merging and if the function is actually doing what you hope to do. https://www.digitalocean.com/community/tutorials/pandas-merge-two-dataframe

**df** now has (after running the cell below):
* filtered out NaN values for (organoid area)
* added 7 new columns by merging with well and metadata: WellID, WellNo, Row, Column, Condition, Concentration, Dye

In [ ]:
df = pd.merge(df,well, on='ImageNumber')
df.rename(columns = {'Image_Metadata_WellID':'WellID'}, inplace = True)
df = pd.merge(df, metadata, on='WellID')
#df.to_excel("after_merge.xlsx")
#display(df)

**one-way ANOVA and Kruskal test on the organoid area for each condition+concentration combination, (did not filter for (number of lumen). did not filter for (spheroid area) yet)**
1. using the **df** dataframe, make separate lists of (organoid area) for each condition+concentration combination, and store them to the variable **areaK**. The best way to understand what areaK is storing is to look at it with display(areaK)
2. use scipy library's one way ANOVA function on **areaK**. we are storing the results to a variable called **oneAnova**. 
3. call display(oneAnova) to see the ANOVA p-value
2. use scipy library's kruskal function on **areaK**. we are storing the results to a variable called **kruskal**
3. use display(kruskal) to look at the kruskal p-value

In [ ]:
areaK = df.groupby(['Condition', 'Concentration'])['spheroids_AreaShape_Area'].apply(list)
display(areaK)

oneAnova = scipy.stats.f_oneway(*areaK)
display(oneAnova)

kruskal = scipy.stats.kruskal(*areaK)
display(kruskal)

**Writing lists of (organoid area) for each condition+concentration (did NOT filter for lumen number, and did NOT filter for area yet) to Excel**

1. get a list of row labels from areaK. store it to an array variable called **row_labels**. One item in **row_labels** contains ('condition','concentration')
2. create a new pandas dataframe called **df_anova**. This will be used to store the **df's** (organoid area) list categorized by condition+concentration
3. for loop. x is the first item in **row_labels**. In **df**, find rows where the (Condition = x's condition) AND (concentration = x's concentration). For those rows, we get their (organoid area). Store that list of (organoid area) values into a list called **nrow**. concatenate **nrow** to **df_anova**. make x be the second item in **row_labels**. repeat the process until we get through the whole row_labels list
4. set the column names of **df_anova** with row_labels

**writing them to specific excel sheet tabs to an excel file called (databaseFile)_Area_SOSRSonly.xlsx **

5. write df_anova dataframe to an excel sheet named '(databaseFile)_Area_SOSRSonly.xlsx', specifically to a tab named "area"

In [ ]:
row_labels = areaK.index.values
print(row_labels)
print(row_labels.size)

df_anova = pd.DataFrame()

for x in row_labels:
    nrow = df.loc[(df['Condition']==x[0]) & (df['Concentration']==x[1])].spheroids_AreaShape_Area
    nrow.reset_index(drop=True, inplace=True)
    df_anova = pd.concat([df_anova, nrow], axis=1, ignore_index=True)
df_anova.columns = row_labels

with pd.ExcelWriter(folder + "/"+databaseFile + '_Area_SOSRSonly.xlsx', engine='xlsxwriter') as writer:
    df_anova.to_excel(writer, sheet_name='area', index=False)

**Filter out the data based on organoid area**
1. make a new dataframe called **df_filter**. copy **df** to **df_filter**
2. using **df_filter**, select rows in which the "spheroids_AreaShape_Area" column is smaller than or equal to a number. update **df_filter**
3. using **df_filter**, select rows in which the "spheroids_AreaShape_Area" column is greater than or equal to a number. update **df_filter**


**df_filter** now has (after running the cell below):
* filtered out NaN values for (organoid area)
* added 7 new columns by merging with well and metadata: WellID, WellNo, Row, Column, Condition, Concentration, Dye
* selected rows in which (organoid area) is in a desired range

In [ ]:
df_filter = df
df_filter = df_filter.loc[df_filter['spheroids_AreaShape_Area'] <= 31415]
df_filter = df_filter.loc[df_filter['spheroids_AreaShape_Area'] >= 7853]
display(df_filter)                 


**write the filtered file to an excel file: (databaseFile)_filtered_SOSRSonly.xlsx**

This file is ready to be sent to the "heatmap_final" Jupyter Notebook file


In [ ]:
df_filter.to_excel(folder + "/"+ databaseFile + "_filtered_SOSRSonly.xlsx")

**create a list of columns that you want to run center/scale on**
1. get a list of all the column names in **df_filter**, store that list into **numList**
2. **dropList** has a list of column names that you **DO NOT** want to run center/scale on
3. for loop. go through each item in **dropList**, and remove that item from **numList**
4. print out the final **numList**. This list contains all the column names that you want to run center/scale on

In [ ]:
numList = df_filter.columns.values.tolist()
# remove additional columns that we don't want

dropList = ['ImageNumber','ObjectNumber','spheroids_Number_Object_Number',
          'WellID','WellNo','Row','Column','Condition','Concentration','Dye']

for c in dropList:
    numList.remove(c)

print(numList)


**Center/scale the data (for all the conditions/concentration groups) and writing it to (databaseFile)_centeredScaled_combined_SOSRSonly.xlsx**
1. make a new dataframe called **df_num**. copy **df** to **df_num**
2. create an object of the StandardScaler() function called **scaler**
3. run sklearn's fit_transform() function on **df_num**, but only on the columns mentioned in **numList**
4. write **df_num** to an excel file called (databaseFile)_centeredScaled_combined_SOSRSonly.xlsx

* we are using the sklearn.preprocessing.StandardScaler function
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
df_num = df_filter
scaler = StandardScaler()
df_num[numList] = scaler.fit_transform(df_num[numList])
df_num.to_excel(folder + "/"+databaseFile + "_centeredScaled_combined_SOSRSonly.xlsx")

**center and scale all the columns (separate processing for each condition) and writing to one excel file for each condition called (databaseFile)\_centeredScaled\_(conditionName)_SOSRSonly.xlsx**
1. create a StandardScaler() object called scaler
2. get a list of unique conditions by running .unique() on the "Conditions" column. store this list to **conditionList**

3. for loop. make variable **x** be the first item in **conditionList**. 
* make df_condCenter a new pandas dataframe. df_condCenter becomes a new empty dataframe very time the loop runs. 
* Set **df_condCenter** to **df_filter**, but only the rows where the Condition column is equal to **x**. 
* Run sklearn's fit_transform() function on df_condCenter, but only on the columns mentioned in **numList**. 
* Write **df_condCenter** to an excel file named (databaseFile)\_centeredScaled\_(conditionName)_SOSRSonly.xlsx
* we finished one round of the for loop. **x** becomes the second item in **conditionList**, and we repeat the commands in the for loop. This continues until all of the items in **conditionList** has been looped through


we are using the sklearn.preprocessing.StandardScaler function
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
scaler = StandardScaler()

conditionList= df_filter['Condition'].unique()
print(conditionList)

#df_combinedCenter = pd.DataFrame()



for x in conditionList:
    df_condCenter = pd.DataFrame()
    df_condCenter = df_filter.loc[(df_filter['Condition']==x)]
    df_condCenter[numList] = scaler.fit_transform(df_condCenter[numList])
    display(df_condCenter)
    df_condCenter.to_excel(folder + "/"+ databaseFile + "_centeredScaled_" + x + "_SOSRSonly.xlsx", index=False)
    #df_combinedCenter = pd.concat([df_combinedCenter, df_condCenter], axis=0, ignore_index=True)

#df_combinedCenter.to_excel(folder + "/"+ databaseFile + "_centeredScaled_" + "combined" + ".xlsx", index=False)
    
